In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time

In [2]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##arbeitsplatop

In [3]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') #arbeit

In [4]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #arbeit

In [ ]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [5]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [6]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [7]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [8]:
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input

In [9]:
#filter all the passes from the synchronized eventdata
passdf=kpimergednew[(kpimergednew['SUBTYPE']=='Pass')]  
passdf.reset_index(drop=True)
#define variables of the two teams
CUID1=passdf.CUID.unique()[0]
CUID2=passdf.CUID.unique()[1]

In [10]:
##get the player ids of the team with club id 1
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()
##get the player ids of the team with club id 2
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [11]:
## initialize a dictionary with the kickoff times for the first and second half
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [12]:
##directionplayframe gives the direction of play of a given team at a specific frame
def directionplayframe(frame,teamid):
    ###Check for teamid
    if teamid == CUID1:
        gk_df=cuid1_team_meta_df.loc[cuid1_team_meta_df['PlayingPositionEnglish']=='goalkeeper'] #select goalkeeper information from team with clubid1
        gks=gk_df['ObjectId'] #select only the ids
        frame1=half['1st half'] #define start of first half
        frame2=half['2nd half'] #define start of second half
        if frame<frame2:  #if the selected frame is smaller than frame2 the frame lies within the first half 
            for items in gks.iteritems():  # iterating through the list of goalkeeper
                if (teamid,items[1],'X') in positions.columns:  #if the id is part of the tracking data column names then continue
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]  ##create a df with the x and y coordinates of the playing goalkeeper
                    if gkpos[teamid,items[1],'X'].mean() < 0:   #if the mean of the x coordinate is smaller than 0 the direction of play is from left to right
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay
    if teamid == CUID2:
        gk_df=cuid2_team_meta_df.loc[cuid2_team_meta_df['PlayingPositionEnglish']=='goalkeeper']
        gks=gk_df['ObjectId']
        frame1=half['1st half']
        frame2=half['2nd half']
        if frame<frame2:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay

In [127]:
##function that returns 1 if the pass is in the direction of play 90 degrees and 0 if not
def qpass90 (row):
    #PUID1=row.PUID1   
    #PUID2=row.PUID2
    t1=int(row.N)
    #t2=int(row.RECFRM)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if directionplayframe(t1,CUID) == 'left to right':
        triangle=np.array([float(52.5),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if directionplayframe(t1,CUID) == 'right to left':
        triangle=np.array([float(-52.5),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [128]:
def qpass90quote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        t1=int(df.loc[df.index[i],'N'])
        CUID=df.loc[df.index[i],'CUID']
        p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
        p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
        if directionplayframe(t1,CUID) == 'left to right':
            triangle=np.array([float(52.5),float(df.loc[df.index[i],'Y_TRACKING'])])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
        if directionplayframe(t1,CUID) == 'right to left':
            triangle=np.array([float(-52.5),float(df.loc[df.index[i],'Y_TRACKING'])])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [129]:
def qpass90goal (row):
    #PUID1=row.PUID1
    #PUID2=row.PUID2
    t1=int(row.N)
    #t2=int(row.RECFRM)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if directionplayframe(t1,CUID) == 'left to right':
        triangle=np.array([float(52.5),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if directionplayframe(t1,CUID) == 'right to left':
        triangle=np.array([float(-52.5),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [130]:
##bestimmt die anzahl der pässe in spielrichtung 90, die nicht erfolgreich waren.
def qpass90goalquote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        t1=int(df.loc[df.index[i],'N'])
        CUID=df.loc[df.index[i],'CUID']
        p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
        p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
        if directionplayframe(t1,CUID) == 'left to right':
            triangle=np.array([float(52.5),0])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
        if directionplayframe(t1,CUID) == 'right to left':
            triangle=np.array([float(-52.5),0])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [140]:
passdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID1)]
passsuccessdf1=passdf1[(passdf1['EVALUATION']=='successfullyComplete') | (passdf1['EVALUATION']=='successful')]
passsuccessrecdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID1)& (kpimergednew.NORECEIVER==False)]
passunsuccessdf1=passdf1[passdf1['EVALUATION']=='unsuccessful']
passdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID2)]
passsuccessdf2=passdf2[(passdf2['EVALUATION']=='successfullyComplete') | (passdf2['EVALUATION']=='successful')]
passsuccessrecdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID2) & (kpimergednew.NORECEIVER==False)]
passunsuccessdf2=passdf2[passdf2['EVALUATION']=='unsuccessful']

In [141]:
#apply the two functions on the dataframe
# runtime around 3 mins !
passsuccessrecdf1['qpass90']=passsuccessrecdf1.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf2['qpass90']=passsuccessrecdf2.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf1['qpass90goal']=passsuccessrecdf1.apply(lambda row:qpass90goal(row),axis=1)
passsuccessrecdf2['qpass90goal']=passsuccessrecdf2.apply(lambda row:qpass90goal(row),axis=1)

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:

In [152]:
cuid190ges=passsuccessrecdf1['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf1,'successful')
cuid290ges=passsuccessrecdf2['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf2,'successful')
cuid190goalges=passsuccessrecdf1['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf1,'successful')
cuid290goalges=passsuccessrecdf2['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf2,'successful')

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [153]:
#Total: Anzahl aller Pässe
#Successful: erfolgreiche Pässe
#Unsuccessful: fehlgeschlagene Pässe
#Successful in Spielrichtung: Erfolgreiche Pässe in Spielrichtung
#Successful gegen Spielrichtung: Alle Pässe, die nicht in Spielrichtung 90 Grad gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung: Alle Pässe, die in Spielrichtung 90 Grad gespielt wurden 
#Successful in Spielrichtung (Tormitte): Erfolgreiche Pässe in Spielrichtung 90 Grad zur Tormitte
#Successful gegen Spielrichtung (Tormitte): Alle Pässe, die nicht in Spielrichtung 90 Grad zur Tormitte gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung (Tormitte): Alle Pässe, die in Spielrichtung 90 Grad zur Tormitte gespielt wurden 
passstatistic = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)

                                           DFL-CLU-000N8Y  DFL-CLU-000N8Z
Total                                                1020             230
Successful                                            942             165
Unsuccessful                                           61              65
Successful in Spielrichtung                           211              47
Successful gegen Spielrichtung                        731             118
Total Pässe in Spielrichtung                          242              99
Successful in Spielrichtung (Tormitte)                227              49
Successful gegen Spielrichtung (Tormitte)             715             116
Total in Spielrichtung (Tormitte)                     265             103


In [156]:
passquotetotal1=passsuccessrecdf1.shape[0]/passdf1.shape[0]
passquotetotal2=passsuccessrecdf2.shape[0]/passdf2.shape[0]
passquote901=passsuccessrecdf1['qpass90'].value_counts()[1]/cuid190ges
passquote902=passsuccessrecdf2['qpass90'].value_counts()[1]/cuid290ges
passquote90goal1=passsuccessrecdf1['qpass90goal'].value_counts()[1]/cuid190goalges
passquote90goal2=passsuccessrecdf2['qpass90goal'].value_counts()[1]/cuid290goalges

In [158]:
#Passquote Total: Erfolgreiche Pässe
#Passquote Total in Spielrichtung: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung gespielt wurden
#Passquote Total in Spielrichtung Tormitte: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung Tormitte gespielt wurden
passstatisticproz = {CUID1: [passquotetotal1,passquote901,passquote90goal1],
                 CUID2: [passquotetotal2,passquote902,passquote90goal2]}
passstatisticprozdf = pd.DataFrame(passstatisticproz,columns= [CUID1, CUID2],
                 index=['Passquote Total','Passquote Total in Spielrichtung','Passquote Total in Spielrichtung Tormitte'])
print(passstatisticprozdf)

                                           DFL-CLU-000N8Y  DFL-CLU-000N8Z
Passquote Total                                  0.923529        0.717391
Passquote Total in Spielrichtung                 0.871901        0.474747
Passquote Total in Spielrichtung Tormitte        0.856604        0.475728
